In [1]:
#Import Packages

import pandas as pd
import numpy as np
import seaborn as sns

import os

%matplotlib inline
pd.set_option('display.max_columns', 500)

# Users

In [2]:
#Import the full data
dfsmall = pd.read_csv("./lastfm-dataset-360k/usersha1-profile.tsv",sep='\t',names=['userId','gender','age','country','signupDate'])
# data = pd.read_csv("./lastfm-dataset-360k/usersha1-artmbid-artname-plays.tsv",sep='\t',names=['userId','artistId','artist','plays'])

## Make sure data makes sense

In [3]:
# Gender
dfsmall.gender.describe()

count     326572
unique         2
top            m
freq      241642
Name: gender, dtype: object

In [4]:
# Age
dfsmall.age.describe()

count    284447.000000
mean         25.098046
std          21.665742
min       -1337.000000
25%          20.000000
50%          23.000000
75%          28.000000
max        1002.000000
Name: age, dtype: float64

In [6]:
dfsmall.loc[(dfsmall['age'] < 0) | (dfsmall['age'] > 122),['age']] = np.nan #assign NaN in place of unrealistic values

In [7]:
dfsmall.age.describe()

count    284388.000000
mean         25.365364
std           9.631621
min           1.000000
25%          20.000000
50%          23.000000
75%          28.000000
max         121.000000
Name: age, dtype: float64

In [8]:
#country
dfsmall.country.describe()

count            359347
unique              239
top       United States
freq              67044
Name: country, dtype: object

In [9]:
#signupdate
dfsmall.loc['signupDate'] = pd.to_datetime(dfsmall['signupDate'],infer_datetime_format=True)
dfsmall.signupDate.describe()

count           359347
unique            2298
top       Jan 24, 2008
freq               569
Name: signupDate, dtype: object

### Missing Values
I don't think we have enough information to find the missing values. So, we need to keep the dataframe as it is.

In [10]:
for column in list(dfsmall.columns):
    print ("{}% of the data from {} column is missing".format(round(dfsmall[column].isnull().sum() * 100 / len(dfsmall[column]),2), column))

0.0% of the data from userId column is missing
9.12% of the data from gender column is missing
20.86% of the data from age column is missing
0.0% of the data from country column is missing
0.0% of the data from signupDate column is missing


# Listening Data

In [11]:
#Import the full data
# dfsmall = pd.read_csv("./lastfm-dataset-360k/usersha1-profile.tsv",sep='\t',names=['userId','gender','age','country','signupDate'])
data = pd.read_csv("./lastfm-dataset-360k/usersha1-artmbid-artname-plays.tsv",sep='\t',names=['userId','artistId','artist','plays'])

In [12]:
data[['userId','artistId','artist']].describe()

,userId,artistId,artist
count,17535655,17309518,17535653
unique,358868,160112,292364
top,13c0df8a58cd3ca9e37d8e90662d2ad3c062eaa5,a74b1b7f-71a5-4011-9441-d0b5e4122711,radiohead
freq,166,77254,77254


In [13]:
data.plays.describe()

count    1.753566e+07
mean     2.151932e+02
std      6.144815e+02
min      0.000000e+00
25%      3.500000e+01
50%      9.400000e+01
75%      2.240000e+02
max      4.191570e+05
Name: plays, dtype: float64

## Missing Values

In [14]:
for column in list(data.columns):
    print ("{}% of the data from {} column is missing".format(round(data[column].isnull().sum() * 100 / len(data[column]),2), column))

0.0% of the data from userId column is missing
1.29% of the data from artistId column is missing
0.0% of the data from artist column is missing
0.0% of the data from plays column is missing


In [15]:
# Let's drop the rows where artistId is missing
data = data.loc[data['artistId'].notnull()]

In [16]:
for column in list(data.columns):
    print ("{}% of the data from {} column is missing".format(round(data[column].isnull().sum() * 100 / len(data[column]),2), column))

0.0% of the data from userId column is missing
0.0% of the data from artistId column is missing
0.0% of the data from artist column is missing
0.0% of the data from plays column is missing


## Merge the datasets

In [17]:
# Merge the data
data = pd.merge(data, dfsmall, on='userId', how='left')
print (len(data))
data.head(2)

17309518


,userId,artistId,artist,plays,gender,age,country,signupDate
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137,f,22.0,Germany,"Feb 1, 2007"
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099,f,22.0,Germany,"Feb 1, 2007"


In [ ]:
directory = 'lastfm-dataset-360K-cleaned'

if not os.path.exists(directory):
    os.makedirs(directory)

#Save Cleaned Data
data.to_csv('./lastfm-dataset-360k-cleaned/merged-data.csv')

In [18]:
# Subset

dfsmall = data.iloc[np.random.choice(data.index,size=10000,replace=False)]
print (data.shape)
print (dfsmall.shape)
dfsmall.head(2)

(17309518, 8)
(10000, 8)


,userId,artistId,artist,plays,gender,age,country,signupDate
249130,03b9c645bc4f578ea1dcb6a975e7ea71fab79da2,a16371b9-7d36-497a-a9d4-42b0a0440c5e,slowdive,108,m,28.0,Germany,"Sep 16, 2006"
16103327,ee48ac7de691d31176a2843d0f2f8f49cab78e6d,5a24bc1a-c093-4a82-84ed-8d7f2da0570d,skinny puppy,407,m,NaN,Canada,"Dec 14, 2006"


In [19]:
directory = 'lastfm-dataset-360k-small'

if not os.path.exists(directory):
    os.makedirs(directory)

dfsmall.to_csv('./lastfm-dataset-360k-small/merged-subset.csv')